In [82]:
%%capture
%run final_chars_classification.ipynb



In [83]:
%run table_detector.ipynb

In [84]:
%run final_functions.ipynb

In [85]:
%run preprocessing.ipynb

In [86]:
%run get_rows_number.ipynb

In [87]:
# initialize_models(True)
load_models()

In [88]:
from tabular import tabular
import math

In [89]:
def solve_expression(img, is_table = False, showTrace = False):
    
    img = cv2.resize(img, (2448, 3264)) # size of A4
    img = img.astype(np.uint8)
    if(is_table):
        # show_images([table_tany(img)])
        # table_tany(img) 
        
        img = table_preprocessing(img, showTrace)
        if showTrace:
            show_images([img])

        cells, col_num, row_num = box_extraction(img, False)#, table= True)

        letters = [classify(i, is_letter=True, is_table=is_table) for i in cells[0:col_num]]
        numbers = [classify(i, is_0_1=True, is_table=is_table) for i in cells[col_num:]]
        
        numbers = [int(i) for i in numbers]
        
        if showTrace:
            print(letters)
            print(numbers)
            print('cols numbers ' + str(col_num))
        
        
        solver = tabular.McCluskey()
        num_outputs = col_num-int(math.log(row_num-1,2))
        print(num_outputs)
        solver.solve(cells=numbers,num_col=col_num,num_outputs=num_outputs)

    else: 

        img = expression_preprocessing(img, showTrace)
        if showTrace:
            show_images([img])

        expression_rows = get_rows_number(img)
        
        if showTrace:
            show_images(expression_rows)
        expressions = [classify(i, is_expression=True, verbose=showTrace) for i in expression_rows ]
        
        if showTrace: print(expressions)
        result = []
        for ex in expressions:
            try:
                ex[1] ='='
                result.append(get_table(ex))
            except:
                result.append('error')
        
        for r in result:
            print(r)

In [93]:
image = cv2.imread("./test_images/ex2.jpg") # TEST R
# image = cv2.imread("./test_images/ex1.jpg")
# image = cv2.imread("./test_images/t2.jpg") # TEST R
# image = cv2.imread("./test_images/exs/t29.jpg") # TEST R

result = solve_expression(image, is_table=False, showTrace=False)




~ B or ~ C 
B and C and D and F 
B implies C 
A xor B 
A xor C 
~ B 
+-----+-----+---------------+
|  B  |  C  |  ~ B or ~ C   |
|-----+-----+---------------|
|  1  |  1  |       0       |
|  1  |  0  |       1       |
|  0  |  1  |       1       |
|  0  |  0  |       1       |
+-----+-----+---------------+
+-----+-----+-----+-----+------------------------+
|  D  |  B  |  F  |  C  |  B and C and D and F   |
|-----+-----+-----+-----+------------------------|
|  1  |  1  |  1  |  1  |           1            |
|  1  |  1  |  1  |  0  |           0            |
|  1  |  1  |  0  |  1  |           0            |
|  1  |  1  |  0  |  0  |           0            |
|  1  |  0  |  1  |  1  |           0            |
|  1  |  0  |  1  |  0  |           0            |
|  1  |  0  |  0  |  1  |           0            |
|  1  |  0  |  0  |  0  |           0            |
|  0  |  1  |  1  |  1  |           0            |
|  0  |  1  |  1  |  0  |           0            |
|  0  |  1  |  0  |  1  |   